# CpGPTPCGrimAge3

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya
import numpy as np

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.CpGPTPCGrimAge3)

class CpGPTPCGrimAge3(pyagingModel):
    def __init__(self):
        super().__init__()

        self.rotation = nn.Parameter(torch.empty((30, 29)), requires_grad=False)

    def preprocess(self, x):
        """
        Scales an array based on the mean and standard deviation.
        """
        mean = torch.tensor(self.preprocess_dependencies[0], device=x.device, dtype=x.dtype)
        std = torch.tensor(self.preprocess_dependencies[1], device=x.device, dtype=x.dtype)
        x = (x - mean) / std
        return x
    
    def forward(self, x):

        x = self.preprocess(x)

        age = x[:, 0].unsqueeze(1)
        proxies = x[:, 1:]

        PCs = torch.mm(proxies, self.rotation)  # Apply PCA rotation

        x = torch.concat([age, PCs], dim=1)

        # Scale
        mean = torch.tensor(self.preprocess_dependencies[2], device=x.device, dtype=x.dtype)
        std = torch.tensor(self.preprocess_dependencies[3], device=x.device, dtype=x.dtype)
        x[:, 1:] = (x[:, 1:] - mean)

In [3]:
model = pya.models.CpGPTPCGrimAge3()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'cpgptpcgrimage3'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2025
model.metadata["approved_by_author"] = '✅'
model.metadata["citation"] = "de Lima Camillo, Lucas Paulo, et al. \"CpGPT: a foundation model for DNA methylation.\" bioRxiv (2024): 2024-10."
model.metadata["doi"] = "https://doi.org/10.1101/2024.10.24.619766"
model.metadata["research_only"] = True
model.metadata["notes"] = None

## Download clock dependencies

In [5]:
logger = pya.logger.Logger()
urls = [
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/cpgpt_grimage3_weights.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/cpgpt_grimage3_weights_all_datasets.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/cpgpt_pcgrimage3_weights.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/input_scaler_mean.npy",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/input_scaler_scale.npy",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/input_scaler_mean_all_datasets.npy",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/input_scaler_scale_all_datasets.npy",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/pca_scaler_mean.npy",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/pca_scaler_scale.npy",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/cpgpt_pcgrimage3_pca_components.npy",
]
dir = "."
for url in urls:
    pya.utils.download(url, dir, logger, indent_level=1)

|-----------> Downloading data to ./cpgpt_grimage3_weights.csv
|-----------> in progress: 100.0000%
|-----------> Downloading data to ./cpgpt_grimage3_weights_all_datasets.csv
|-----------> in progress: 100.0000%
|-----------> Downloading data to ./cpgpt_pcgrimage3_weights.csv
|-----------> in progress: 100.0000%
|-----------> Downloading data to ./input_scaler_mean.npy
|-----------> in progress: 100.0000%%
|-----------> Downloading data to ./input_scaler_scale.npy
|-----------> in progress: 100.0000%%
|-----------> Downloading data to ./input_scaler_mean_all_datasets.npy
|-----------> in progress: 100.0000%%
|-----------> Downloading data to ./input_scaler_scale_all_datasets.npy
|-----------> in progress: 100.0000%%
|-----------> Downloading data to ./pca_scaler_mean.npy
|-----------> in progress: 100.0000%%
|-----------> Downloading data to ./pca_scaler_scale.npy
|-----------> in progress: 100.0000%%
|-----------> Downloading data to ./cpgpt_pcgrimage3_pca_components.npy
|-----------

## Load features

#### From CSV

In [6]:
df = pd.read_csv('cpgpt_pcgrimage3_weights.csv')
model.features = pd.read_csv('cpgpt_grimage3_weights.csv')['feature'].tolist()

In [7]:
df.head()

,feature,coefficient
0,age,0.672689
1,pca_component_1,0.759243
2,pca_component_2,-0.686806
3,pca_component_3,0.233157
4,pca_component_4,-0.138548


## Load weights into base model

In [8]:
pc_matrix = np.load('cpgpt_pcgrimage3_pca_components.npy').T

#### PC Components

In [9]:
model.rotation.data = torch.tensor(pc_matrix).float()

#### Linear model

In [10]:
weights = torch.tensor(df['coefficient'].tolist()).unsqueeze(0)
intercept = torch.tensor([0.0])

#### Linear model

In [11]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [12]:
scale_mean = np.load('input_scaler_mean.npy')
scale_std = np.load('input_scaler_scale.npy')

pca_scale_mean = np.load('pca_scaler_mean.npy')
pca_scale_std = np.load('pca_scaler_scale.npy')

model.reference_values = None

## Load preprocess and postprocess objects

In [13]:
model.preprocess_name = 'scale'
model.preprocess_dependencies = [scale_mean, scale_std, pca_scale_mean, pca_scale_std]

In [14]:
model.postprocess_name = 'cox_to_years'
model.postprocess_dependencies = [
    4.66184408e-17,
    1.70884158624939,
    58.8234007654456,
    13.091231557630831
]

## Check all clock parameters

In [15]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '✅',
 'citation': 'de Lima Camillo, Lucas Paulo, et al. "CpGPT: a foundation model '
             'for DNA methylation." bioRxiv (2024): 2024-10.',
 'clock_name': 'cpgptpcgrimage3',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.1101/2024.10.24.619766',
 'notes': None,
 'research_only': True,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2025}
reference_values: None
preprocess_name: 'scale'
preprocess_dependencies: [array([ 5.88234008e+01,  3.37892988e+04,  2.13030677e+01,  6.29543290e-02,
        1.71182803e+06,  3.48032879e+02,  8.07863072e+03,  8.40183045e+02,
       -4.62783972e-01, -2.81553566e-01,  2.08716775e-02, -2.03755490e-01,
       -3.31462601e-01, -1.99447326e-01, -2.86641682e-01,  9.69636186e-02,
        9.23084436e-02, -1.77190600e-01, -5.19144939e-01,  2.77395512e-01,
       -5.05034895e-02, -1.6

## Basic test

In [16]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float).double()
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[-365.0296],
        [-418.9304],
        [-114.4988],
        [ -79.3617],
        [ 576.5590],
        [ 419.1323],
        [-391.7414],
        [ 237.9895],
        [-340.7663],
        [-172.7751]], dtype=torch.float64, grad_fn=<AddBackward0>)

## Save torch model

In [17]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [18]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: input_scaler_mean_all_datasets.npy
Deleted file: cpgpt_pcgrimage3_pca_components.npy
Deleted file: cpgpt_pcgrimage3_weights.csv
Deleted file: input_scaler_scale_all_datasets.npy
Deleted file: input_scaler_scale.npy
Deleted file: pca_scaler_mean.npy
Deleted file: cpgpt_grimage3_weights_all_datasets.csv
Deleted file: pca_scaler_scale.npy
Deleted file: cpgpt_grimage3_weights.csv
Deleted file: input_scaler_mean.npy
